#### Step 1: Define ontology 

#### Step 2: Populate KG 

#### Namespace 

@db: https://www.dbpedia.org/

db:Football_player
db:name 
db:height 

db:position


In [ ]:
# Add classes and properties 

# Populate KG 